# DeepLOB: Deep Convolutional Neural Networks for Limit Order Books

This jupyter notebook is used to reconstruct a paper [2] published in IEEE Transactions on Singal Processing. I use FI-2010 [1] dataset and present how model architecture is constructed here. The dataset can be downloaded from: https://etsin.fairdata.fi/dataset/73eb48d7-4dbc-4a10-a52a-da745b47a649

[1] Ntakaris A, Magris M, Kanniainen J, Gabbouj M, Iosifidis A. Benchmark dataset for mid‐price forecasting of limit order book data with machine learning methods. Journal of Forecasting. 2018 Dec;37(8):852-66. https://arxiv.org/abs/1705.03233

[2] Zhang Z, Zohren S, Roberts S. DeepLOB: Deep convolutional neural networks for limit order books. IEEE Transactions on Signal Processing. 2019 Mar 25;67(11):3001-12. https://arxiv.org/abs/1808.03668

## About the dataset

Approximately four million events for ten consecutive trading days for five stocks.

## Basic set-up

In [1]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.random import set_seed
from keras.utils import np_utils
from keras.models import load_model, Model
from keras.optimizers import Adam
from keras.layers import Flatten, Dense, Dropout, Activation, Input, Reshape, Conv2D, MaxPooling2D, LeakyReLU, concatenate
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

# set random seeds
np.random.seed(1)
set_seed(2)

# limit gpu usage for keras - WHY?
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

## Data preparation

I will use the No Auction half of the dataset. 

The dataset offers multiple normalisation options, I use the z-score normalisation. 

The dataset is already split into training data and testing data. I use the first 6 days as training data, the 7th day as validation data and the last 3 days as testing data.

The first 40 columns of the FI-2010 dataset are 10 levels of ask and bid information for a limit order book and I only use these 40 features in our network. The last 5 columns of the FI-2010 dataset are the labels with different prediction horizons (up, down, stationary). 

In [2]:
def trim(data):
    df = data[:40, :].T
    return df

def get_labels(data):
    df = data[-5:, :].T
    return df

def classification(data, labels, timestep): # train_trimmed, train_labels, 100
    [N, D] = data.shape # Shape of trimmed features data [200,000, 40]
    df = np.array(data) # Turn it into an array

    dY = np.array(labels) # Turn labels data into an array

    dataY = dY[timestep - 1:N] # Trim labels data [] 

    dataX = np.zeros((N - timestep + 1, timestep, D))  #[200,000 -99, 100, 40]
    for i in range(timestep, N + 1): #[100, 200,000 + 1]
        dataX[i - timestep] = df[i - timestep:i, :]

    return dataX.reshape(dataX.shape + (1,)), dataY # explain

In [3]:
data_path = 'E:/New Downloads/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore'

train_raw = np.loadtxt(data_path + '/NoAuction_Zscore_Training/Train_Dst_NoAuction_ZScore_CF_6.txt')
validation_raw = np.loadtxt(data_path + '/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_6.txt')
test1_raw = np.loadtxt(data_path + '/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_7.txt')
test2_raw = np.loadtxt(data_path + '/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_8.txt')
test3_raw = np.loadtxt(data_path + '/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_9.txt')
test_raw = np.hstack((test1_raw, test2_raw, test3_raw))

In [4]:
# Extract relevant features from dataset
train_trimmed = trim(train_raw)
validation_trimmed = trim(validation_raw)
test_trimmed = trim(test_raw)

# Extract labels from dataset
train_labels = get_labels(train_raw)
validation_labels = get_labels(validation_raw)
test_labels = get_labels(test_raw)

# Prepare all data - create an array containing the values of all features for last 100 timesteps
train_x, train_y = classification(train_trimmed, train_labels, 100)
train_y = train_y[:,3] - 1 # Prediction horizon set to 3 events in the future, and labels are rescaled
train_y = np_utils.to_categorical(train_y, 3)

validation_x, validation_y = classification(validation_trimmed, validation_labels, 100)
validation_y = validation_y[:,3] - 1 # Prediction horizon set to 3 events in the future, and labels are rescaled
validation_y = np_utils.to_categorical(validation_y, 3)

test_x, test_y = classification(test_trimmed, test_labels, 100)
test_y = test_y[:,3] - 1 # Prediction horizon set to 3 events in the future, and labels are rescaled
test_y = np_utils.to_categorical(test_y, 3)

## Creating the model architecture

Details of the architecture can be found in [2]. Briefly:



In [7]:
def create_model(timestep, no_features, no_lstm):
    input_layer = Input(shape=(timestep, no_features, 1))
    
    # Convolutional block - COMPLETE
    conv_first1 = Conv2D(32, (1, 2), strides=(1, 2))(input_layer)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)

    conv_first1 = Conv2D(32, (1, 2), strides=(1, 2))(conv_first1)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)

    conv_first1 = Conv2D(32, (1, 10))(conv_first1)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)
    
    # Inception module - COMPLETE
    convsecond_1 = Conv2D(64, (1, 1), padding='same')(conv_first1)
    convsecond_1 = LeakyReLU(alpha=0.01)(convsecond_1)
    convsecond_1 = Conv2D(64, (3, 1), padding='same')(convsecond_1)
    convsecond_1 = LeakyReLU(alpha=0.01)(convsecond_1)

    convsecond_2 = Conv2D(64, (1, 1), padding='same')(conv_first1)
    convsecond_2 = LeakyReLU(alpha=0.01)(convsecond_2)
    convsecond_2 = Conv2D(64, (5, 1), padding='same')(convsecond_2)
    convsecond_2 = LeakyReLU(alpha=0.01)(convsecond_2)

    convsecond_3 = MaxPooling2D((3, 1), strides=(1, 1), padding='same')(conv_first1)
    convsecond_3 = Conv2D(64, (1, 1), padding='same')(convsecond_3)
    convsecond_3 = LeakyReLU(alpha=0.01)(convsecond_3)
    
    convsecond_output = concatenate([convsecond_1, convsecond_2, convsecond_3], axis=3)
    
    # use the MC dropout here - COMPLETE
    
    conv_reshape = Reshape((int(convsecond_output.shape[1]), int(convsecond_output.shape[3])))(convsecond_output)
    
    # LSTM layer
    conv_lstm = CuDNNLSTM(no_lstm)(conv_reshape) # CuDNN is much faster than vanilla LSTM when run on GPU
    
    # Output layer
    output_layer = Dense(3, activation='softmax')(conv_lstm)
    
    # Putting layers together
    model = Model(inputs = input_layer, outputs = output_layer)
    optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1) # Explain hyperparameters
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) # Explain
    
    return model

In [8]:
deeplob = create_model(100, 40, 64)
deeplob.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 40, 1)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 100, 20, 32)  96          input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 100, 20, 32)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 100, 20, 32)  4128        leaky_re_lu[0][0]                
_______________________________________________________________________________________

## Explain hyperparamters 

## Training the model

In [ ]:
deeplob.fit(train_x, train_y, epochs=5, batch_size=32, verbose=2, validation_data=(validation_x, validation_y))

## Saving the model

In [ ]:
# Serialize model to JSON
deeplob_json = deeplob.to_json()
with open('deeplob_json', 'w') as json_file:
    json_file.write(deeplob_json)

# Serialize weights to HDF5
deeplob.save_weights("deeplob.h5")

print("Model saved to disk once")

deeplob.save("C:/Users/hamza/Downloads/DeepLOB/model")

print("Model saved to disk twice")